In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [46]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

In [47]:
# Lee GRD del INT
df_egresos = pd.read_csv("../data/processed/df_procesada.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_egresos["FECHA_INGRESO"] = pd.to_datetime(df_egresos["FECHA_INGRESO"])
df_egresos["FECHA_EGRESO"] = pd.to_datetime(df_egresos["FECHA_EGRESO"])

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_56595/245708600.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_egresos = pd.read_csv("../data/processed/df_procesada.csv", usecols=COLUMNAS_GRD)


In [48]:
grd_y_pabellon = df_egresos.merge(df_pabellon, how="inner", left_on="RUT", right_on="ID_PACIENTE")

In [49]:
grd_y_pabellon

,RUT,SEXO,PREVI,BENEF,NOM_HOPI_P,DIAS_ESTAD,DIAG1,DIAG2,DIAG3,INTERV_Q,COD_FONASA,T_DIAG1,T_DIAG2,T_DIAG3,INTERV_Q_PPAL,INTERV_Q_2,INTERV_Q_3,PROCED,PROCED_PPAL,PROCED_2,PROCED_3,FECHA_INGRESO,FECHA_EGRESO,fecha,sexo,ID_PACIENTE,edad,prev,especialidad,primer_diagnostico,segundo_diagnostico,nombre_de_la_operacion,reintervencion_no_prog,codigo_i,codigo_ii,ppv_-ges,tipo_de_cirugia,h_entrada,h_inicio,h_termino,h_salida,duracion,aseo,n_pabellon,sevicio,combinacion_operaciones,cirugia_programada_o_urgente
0,8fd17789a38d8bf4150d6983c757fcb2c63d5aed240765...,1,1,B,NaN,33,I052,NaN,NaN,1,1703061.0,Estenosis mitral con insuficiencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-30,2016-05-02,2016-04-06,M,8fd17789a38d8bf4150d6983c757fcb2c63d5aed240765...,74,B,CV,C. ISQUEMICA - ENF 1 VASO,IM SEVERA - DISFUNCION SEVERA VI,REVASC MIOCARDICA (LIMA-DA) - RVA (BIO 29 ),NO,1703061,0000000,1703461,P,17:35:00,19:00:00,00:00:00,00:10:00,06:35:00,00:30:00,1,UCI,1703061_0000000,Programada
1,dbf339bf59332c2b38bba08ac2eb2ff15414f6d753eb8a...,1,1,B,Hospital Presidente Carlos Ibáñez del Campo (L...,33,Q233,NaN,NaN,1,1703153.0,Insuficiencia mitral congénita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-21,2016-05-24,2016-04-28,H,dbf339bf59332c2b38bba08ac2eb2ff15414f6d753eb8a...,22,B,CV,CANAL AV PARCIAL - EBSA CENTRAL C/ INSUF SEVERA,INSUF TRICUSP MOD - SD DOWN,RECAMBIO VALV MITRAL - CIERRE CIA OP - PLASTIA...,NO,1703061,1701055,1703466,E,08:20:00,09:40:00,14:55:00,15:15:00,06:55:00,00:30:00,2,UCI,1703061_1701055,Programada
2,dbf339bf59332c2b38bba08ac2eb2ff15414f6d753eb8a...,1,1,B,Hospital Presidente Carlos Ibáñez del Campo (L...,33,Q233,NaN,NaN,1,1703153.0,Insuficiencia mitral congénita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-21,2016-05-24,2016-05-03,H,dbf339bf59332c2b38bba08ac2eb2ff15414f6d753eb8a...,22,B,CV,SD DOWN EBSA CIA Y CIV,NaN,INSTALACION DE CATATER GROSHONG,NO,402032,0000000,0000000,E,14:20:00,14:25:00,15:05:00,15:25:00,01:05:00,00:30:00,3,UTI CV,402032_0000000,Programada
3,dbf339bf59332c2b38bba08ac2eb2ff15414f6d753eb8a...,1,1,B,Hospital Presidente Carlos Ibáñez del Campo (L...,33,Q233,NaN,NaN,1,1703153.0,Insuficiencia mitral congénita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-21,2016-05-24,2016-05-12,H,dbf339bf59332c2b38bba08ac2eb2ff15414f6d753eb8a...,22,B,MCP,CANAL AV COMPLETO OP X EBSA,RAV POST,NaN,NO,1703253,0000000,0000000,E,09:00:00,09:45:00,11:00:00,11:30:00,02:30:00,00:30:00,2,UTI CV,1703253_0000000,Programada
4,454eb6927cd058c20b02b99a874f8de0c0ad68f80ffc5b...,1,1,D,"Hospital Dr. Luis Tisné B. (Santiago, Peñalolén)",13,I251,NaN,NaN,1,1703061.0,Enfermedad aterosclerótica del corazón,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-11,2016-05-24,2016-05-17,H,454eb6927cd058c20b02b99a874f8de0c0ad68f80ffc5b...,53,D,CV,ENF CORONARIA,HTA TABAQUISMO DISLIPIDEMIA,REVASC MIOCARDICA,NO,1703061,0000000,1730461,E,12:20:00,13:40:00,17:00:00,17:15:00,04:55:00,00:30:00,2,UCI,1703061_0000000,Programada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41652,9d6b24b603248f37858fab5e6f82d67e86679924146a88...,2,1,B,Hospital Clínico Universidad de Chile,23,I251,NaN,I252,1,NaN,ENFERMEDAD ATEROSCLEROTICA DEL CORAZÓN,NaN,INFARTO ANTIGUO DEL MIOCARDIO,1703061.0,NaN,NaN,2.0,NaN,NaN,NaN,2021-04-19,2021-05-12,2021-04-26,M,9d6b24b603248f37858fab5e6f82d67e86679924146a88...,76,B,CV,ENFERMEDAD CORONARIA,NaN,REVASC MIOCARDICA,NO,1703061,0000000,1703461,E,08:00:00,09:20:00,11:55:00,12:10:00,04:10:00,00:30:00,2,UCI,1703061_0000000,Programada
41653,699537c689c7869a4401469f28dbedac6949569b4f12e5...,1,1,B,Hospital Clínico Regional Dr. Guillermo Grant ...,149,U071,NaN,J150,1,NaN,"COVID-19, VIRUS IDENTIFICADO",NaN,NEUMONIA DEBIDA A KLEBSIELLA PNEUMONIAE,1302072.0,1704018.0,NaN,2.0,NaN,NaN,NaN,2020-12-14,2021-05-12,2021-01-02,H,699537c689c7869a4401469f28dbedac6949569b4f12e5...,71,B,CT,COLECISTITIS AGUDA,NaN,COLECISTECTOMIA ABIERTA,NO,170401